In [1]:
from tree_dataset import TreeDataset
import model as m
from torch.utils.data import DataLoader
import os
import torchvision.transforms as transforms
from d2lvit import *
import copy
import torch.nn as nn

In [2]:
preprocess = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor()
])
train_set = TreeDataset(os.path.join('..', 'data', 'extra_variety_4k'), preprocess) 
val_set = TreeDataset(os.path.join('..', 'data', 'extra_variety_2k'), preprocess)
print(f'Train size: {len(train_set)} Val size: {len(val_set)}')
train_loader = DataLoader(train_set, batch_size=32)
val_loader = DataLoader(val_set, batch_size=32)
device = m.get_device()
config = {'labels_key': 'digit_labels'}

Train size: 4000 Val size: 2000
Identified CUDA device: NVIDIA GeForce RTX 3060


In [3]:
model = torch.load(os.path.join('..', 'models', 'd2lvit_3', 'digit-model.pt'))

In [4]:
train_acc = m.predict(model, train_loader, device, config, None)
print(train_acc)

1.0


In [5]:
val_acc = m.predict(model, val_loader, device, config, None)
print(val_acc)

1.0


In [6]:
digits_model = copy.deepcopy(model)

In [7]:
model.head = nn.Sequential(
    nn.LayerNorm((512,), eps=1e-05, elementwise_affine=True),
    nn.Linear(512, 45)
)
model = model.to(device)

In [8]:
model(torch.unsqueeze(train_set[0]['image'], 0).to(device))

tensor([[-0.8718, -0.6324, -0.1467,  0.2002,  0.2749,  0.4353, -0.1071, -0.6023,
          0.2261,  0.6045, -0.6051, -0.6577, -0.3231,  0.2476,  0.4779, -0.5069,
         -0.1710, -0.6678, -0.2356, -0.6726,  1.0237,  0.4148, -0.6056,  0.5098,
         -0.0053,  0.9754,  0.2620, -0.1762,  0.7682,  0.3814, -0.4088, -0.0744,
         -0.3448, -0.2181,  0.7172, -0.1708, -0.0571, -0.0330, -0.6256,  0.6114,
          0.5187, -0.6235,  0.4866, -0.4230, -0.6395]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

In [9]:
m.train(model, 0.0001, 0, 100, train_loader, val_loader, device, os.path.join('..', 'models', 'd2lvit_3'), digits_model)

Epoch 10 done, train loss: 0.0023 val acc: 1.0000
Epoch 20 done, train loss: 0.0015 val acc: 1.0000
Epoch 30 done, train loss: 0.0003 val acc: 1.0000
Epoch 40 done, train loss: 0.0003 val acc: 1.0000
Epoch 50 done, train loss: 0.0002 val acc: 1.0000
Epoch 60 done, train loss: 0.0011 val acc: 1.0000
Epoch 70 done, train loss: 0.0001 val acc: 1.0000
Epoch 80 done, train loss: 0.0003 val acc: 1.0000
Epoch 90 done, train loss: 0.0027 val acc: 1.0000
Epoch 100 done, train loss: 0.0001 val acc: 1.0000


In [10]:
model = torch.load(os.path.join('..', 'models', 'd2lvit_3', 'tree-model.pt'))

In [11]:
config = {'labels_key': 'tree_label'}
train_acc = m.predict(model, train_loader, device, config, digits_model)
print(train_acc)

1.0


In [12]:
val_acc = m.predict(model, val_loader, device, config, digits_model)
print(val_acc)

1.0
